# ollama on collab

[2 Ways to Run DeepSeek R1 on Google Colab Free Tier with Ollama ](https://dev.to/0xkoji/2-ways-to-run-ollama-on-google-colab-free-tier-3i4) method 1

## install ollama

In [1]:
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  36528      0 --:--:-- --:--:-- --:--:-- 36586
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Get:4 http://security.ubuntu.com/

In [ ]:
!curl https://ollama.ai/install.sh | sh

## start server

In [2]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [3]:
OLLAMA_MODEL = "jobautomation/OpenEuroLLM-Romanian:latest"

In [4]:
!ollama pull {OLLAMA_MODEL}

In [5]:
!ollama ls

NAME                                         ID              SIZE      MODIFIED       
jobautomation/OpenEuroLLM-Romanian:latest    08d230e07f63    8.1 GB    39 seconds ago    


In [6]:
!pip install ollama

In [7]:
import ollama

response = ollama.chat(model=OLLAMA_MODEL, messages=[
  {
    'role': 'user',
    'content': "Cate degete are o mana?",
  },
])

print(response['message']['content'])

O mână are cinci degete. Acestea sunt: degetul mare, arătătorul, mijlocul, inelarul și degetul mic.



In [8]:
!ollama ps

NAME                                         ID              SIZE     PROCESSOR          UNTIL              
jobautomation/OpenEuroLLM-Romanian:latest    08d230e07f63    12 GB    82%/18% CPU/GPU    4 minutes from now    


In [10]:
!nvidia-smi

Sun Jul 27 16:44:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   60C    P0             30W /   70W |     466MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
!pip install gradio -qqq

In [12]:
import gradio as gr

def chat_with_ollama(message, history):
    messages = []

    # Add previous conversation history
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})

    # Add the current user message
    messages.append({"role": "user", "content": message})

    try:
        response = ollama.chat(model=OLLAMA_MODEL, messages=messages, stream=True)

        # Stream the response
        full_response = ""
        for chunk in response:
            if 'message' in chunk and 'content' in chunk['message']:
                content = chunk['message']['content']
                full_response += content
                yield full_response # Yield the accumulating response for streaming

    except Exception as e:
        return f"Error: {e}"

In [13]:
iface = gr.ChatInterface(
    fn=chat_with_ollama,

    chatbot=gr.Chatbot(
        show_copy_button=True,
        resizable=True,
    ),
    title="Ollama Chat",

    examples=["Câte degete sunt la o mână?"],
)

/tmp/ipython-input-13-173987419.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


In [14]:
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d80b9cd11eb72f6b36.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
